In [1]:
import csv
import re
import os
import shutil
import numpy as np
import pandas as pd
from zipfile import ZipFile
import string
import PyPDF2
import textract

In [2]:
from utils import *

In [944]:
basepath = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Platform_Records/WFPs/'
source = 'DOFST-K_SBE43F_SN_3523_Calibration_2018'

In [945]:
def extract_pdf(pdf):
    
    ptext = textract.process(pdf, encoding='utf-8')
    ptext = ptext.decode('utf-8')
    
    for line in ptext.splitlines():
        if '(adj)' in line:
            coef, _, val, _ = line.split()
            coef = coef.strip()
            val = val.strip()
            continue
            
        if 'CALIBRATION DATE' in line:
            _, caldate = line.split(':')
            caldate = pd.to_datetime(caldate).strftime('%Y%m%d')
            continue
            
    return coef, val, caldate

In [946]:
for root, dirs, files in os.walk(basepath):
    for name in files:
        if source in name and not name.endswith('.v'):
            filepath = os.path.join(root,name)
            print(filepath)

/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Platform_Records/WFPs/Coastal_WFPs/14300-02/DOFST-K_SBE43F_SN_3523_Calibration_2018-05-25.zip


In [947]:
temp_directory = '/'.join((os.getcwd(),'temp'))
# Check if the temp directory exists; if it already does, purge and rewrite
if os.path.exists(temp_directory):
    shutil.rmtree(temp_directory)
    ensure_dir(temp_directory)
else:
    ensure_dir(temp_directory)

In [948]:
if filepath.endswith('.zip'):
    with ZipFile(filepath) as zfile:
        zfile.extractall(path=temp_directory)

In [960]:
cal = [name for name in os.listdir(temp_directory) if name.endswith('.cal')][0]
pdf = [name for name in os.listdir(temp_directory) if name.endswith('.pdf') and 'SOC' in name][0]

In [961]:
pdf

'SBE 43F O3523 SOCAdjusted 25May18.pdf'

In [962]:
temp_directory

'/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Metadata_Review/temp'

In [963]:
with open(temp_directory+'/'+cal) as file:
    data = file.read()

In [964]:
for line in data.splitlines():
    print(line)

INSTRUMENT_TYPE=SBE43F
SERIALNO=3523
OCALDATE=25-May-18
SOC= 3.365236e-004
FOFFSET=-8.250858e+002
A=-3.430888e-003
B= 1.642034e-004
C=-2.967717e-006
E= 3.600000e-002
Tau20= 8.100000e-001


In [965]:
coefficient_name_map = {
    'FOFFSET':'CC_frequency_offset',
    'Soc':'CC_oxygen_signal_slope',
    'A':'CC_residual_temperature_correction_factor_a',
    'B':'CC_residual_temperature_correction_factor_b',
    'C':'CC_residual_temperature_correction_factor_c',
    'E':'CC_residual_temperature_correction_factor_e'
}
calibration_coeffs = {}

In [966]:
for line in data.splitlines():
    
    
    coef, val = line.split('=')
    coef = coef.strip()
    val = val.strip()
    
    if coef == 'SERIALNO':
        sn = val
        continue
    
    name = coefficient_name_map.get(coef)
    
    if name is not None:
        calibration_coeffs.update({name: float(val)})   
        
coef, val, caldate = extract_pdf(temp_directory+'/'+pdf)
name = coefficient_name_map.get(coef)
if name is not None:
    calibration_coeffs.update({name: float(val)})

In [967]:
pdf

'SBE 43F O3523 SOCAdjusted 25May18.pdf'

In [968]:
calibration_coeffs

{'CC_frequency_offset': -825.0858,
 'CC_residual_temperature_correction_factor_a': -0.003430888,
 'CC_residual_temperature_correction_factor_b': 0.0001642034,
 'CC_residual_temperature_correction_factor_c': -2.967717e-06,
 'CC_residual_temperature_correction_factor_e': 0.036,
 'CC_oxygen_signal_slope': 0.00035762}

In [969]:
wfp, filename = filepath.split('/')[-2::]
source = f'Date in filename from {pdf}. Source file: {wfp} > {filename} > {cal} and {pdf}.'

In [970]:
# Create a dataframe to write to the csv
data = {
    'serial': ['43' + '-' + sn]*len(calibration_coeffs),
    'name': list(calibration_coeffs.keys()),
    'value': list(calibration_coeffs.values())
}
df = pd.DataFrame().from_dict(data)
# Now merge the coefficients dataframe with the notes
df['notes'] = ''
# Add in the source file
df['notes'].iloc[0] = df['notes'].iloc[0] + ' ' + source
# Sort the data by the coefficient name
df = df.sort_values(by='name')

/home/andrew/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [971]:
df

,serial,name,value,notes
0,43-3523,CC_frequency_offset,-825.085800,Date in filename from SBE 43F O3523 SOCAdjust...
5,43-3523,CC_oxygen_signal_slope,0.000358,
1,43-3523,CC_residual_temperature_correction_factor_a,-0.003431,
2,43-3523,CC_residual_temperature_correction_factor_b,0.000164,
3,43-3523,CC_residual_temperature_correction_factor_c,-0.000003,
4,43-3523,CC_residual_temperature_correction_factor_e,0.036000,


In [972]:
savefile = 'CGINS-DOFSTK-' + str(sn).zfill(5) + '__' + caldate + '.csv'
savefile

'CGINS-DOFSTK-03523__20180525.csv'

In [973]:
df.to_csv(temp_directory+'/'+savefile)

**====================================================================================================================**
Now can import the csv file for comparison

In [974]:
csv_name = 'calibration/DOFSTK/' + savefile

In [975]:
csv_file = pd.read_csv('/home/andrew/Documents/OOI-CGSN/asset-management/' + csv_name)

In [976]:
source_file = pd.read_csv(temp_directory + '/' + savefile)

In [977]:
csv_file

,serial,name,value,notes
0,43-3523,CC_frequency_offset,-825.085800,date in filename comes from Soc-adjusted calda...
1,43-3523,CC_oxygen_signal_slope,0.000358,adjusted Soc (O2 signal slope) value
2,43-3523,CC_residual_temperature_correction_factor_a,-0.003431,NaN
3,43-3523,CC_residual_temperature_correction_factor_b,0.000164,NaN
4,43-3523,CC_residual_temperature_correction_factor_c,-0.000003,NaN
5,43-3523,CC_residual_temperature_correction_factor_e,0.036000,NaN


In [978]:
source_file.drop(columns='Unnamed: 0', inplace=True)
source_file

,serial,name,value,notes
0,43-3523,CC_frequency_offset,-825.085800,Date in filename from SBE 43F O3523 SOCAdjust...
1,43-3523,CC_oxygen_signal_slope,0.000358,NaN
2,43-3523,CC_residual_temperature_correction_factor_a,-0.003431,NaN
3,43-3523,CC_residual_temperature_correction_factor_b,0.000164,NaN
4,43-3523,CC_residual_temperature_correction_factor_c,-0.000003,NaN
5,43-3523,CC_residual_temperature_correction_factor_e,0.036000,NaN


In [979]:
csv_file == source_file

,serial,name,value,notes
0,True,True,True,False
1,True,True,True,False
2,True,True,True,False
3,True,True,True,False
4,True,True,True,False
5,True,True,False,False


In [980]:
csv_file['value'].iloc[5], source_file['value'].iloc[5]

(0.036, 0.036000000000000004)

In [943]:
for line in ptext.splitlines():
    print(line)

NameError: name 'ptext' is not defined

In [726]:
for line in data.splitlines():
    if '=' in line:
        coef, val, *ignore = line.split('=')
        if coef.strip() in coefficient_name_map.keys():
            calibration_coeffs.update({coefficient_name_map.get(coef.strip()):float(val)})
            
    if 'oxygen S/N' in line:
        sn = line[line.find('=')+len('='):line.rfind(':')].strip()

AttributeError: 'dict' object has no attribute 'splitlines'

In [ ]:
calibration_coeffs

In [ ]:
sn